## 1. Importy i ustawienie seedu losowości

W pierwszym bloku importujemy potrzebne biblioteki:
- `torch` - biblioteka PyTorch do obliczeń tensorowych
- `tenseal` – biblioteka do szyfrowania homomorficznego
- `pandas` – obsługa danych
- `random, np.random, torch.random` – generowanie liczb losowych
- `time` – mierzenie czasu wykonania operacji
- `StandardScaler` – normalizacja danych
- `numpy` – operacje matematyczne
- `matplotlib.pyplot` – wizualizacja danych
- `datasets` i `train_test_split` – dostęp do gotowych zbiorów danych i ich podział na zbiór treningowy i testowy

Następnie ustawiamy seed losowości w trzech różnych bibliotekach (torch, random, numpy). To zapewnia powtarzalność wyników – np. przy każdym uruchomieniu programu generowane liczby losowe będą takie same. To istotne w analizach naukowych lub przy testowaniu modeli ML.

## 2. Wczytanie zbioru danych i podział na zbiór treningowy i testowy

W tym bloku wczytujemy gotowy zbiór danych dotyczących raka piersi z biblioteki `scikit-learn`. Zmienna `X` to dane wejściowe, kolumny takie jak np. „mean radius”, „mean texture”, itp. Zmienna `y_np` to etykiety klas: 0 oznacza nowotwór złośliwy, 1 – łagodny. Następnie dzielimy dane na zbiór treningowy oraz testowy w proporcjach: 67% - zbiór treningowy, 33% - zbiór testowy. `Random state = 42` zapewnia powtarzalność tego podziału.

## 3. Konwersja danych do tensorów PyTorch

Dane wejściowe `X_train_np` i `X_test_np` są konwertowane do tensorów typu `float32`. Etykiety `y_train_np` i `y_test_np` również są konwertowane, ale dodatkowo za pomocą `unsqueeze` zmieniamy ich kształt z `[n]` na `[n,1]`, w celu dopasowania ich do sieci neuronowych, wymagają kolumnowej formy danych wyjściowych.

## 4. Standaryzacja danych

Dla każdej cechy w danych obliczana jest średnia `mean` i odchylenie standardowe `std` w zbiorze treningowym. Dla cech, gdzie odchylenie standardowe wynosi 0, zostaje ono zastąpione 1.0, aby uniknąć dzielenia przez zero. Dane są następnie standaryzowane; `x_test` jest skalowane tymi samymi wartościami `mean` i `std`, co `x_train`.


Podsumowując dane z klasyfikacji raka piersi zostały pobrane, podzielone, skonwertowane do tensorów PyTorch i odpowiednio przeskalowane.

## 5. Wizualizacja danych treningowych

Tworzymy wykres rozrzutu punktów treningowych, bazujący na pierwszych dwóch cechach. Kolor każdego punktu odpowiada klasie (0 lub 1), więc wizualnie możemy sprawdzić, czy dane klas są łatwe do rozdzielenia. Cechy są wcześniej standaryzowane, więc mają średnią 0 i podobną skalę. Wykres pomaga zrozumieć rozkład danych oraz ich separowalność.

#### Obserwacje na podstawie wykresu:

- Separacja klas nie jest wyraźna: nie widać jednej prostej linii, która łatwo oddzieliłaby klasę 0 od 1 tylko na podstawie tych dwóch cech. Części klas nachodzą na siebie, co może prowadzić do trudności w klasyfikacji przy prostym modelu
- Gęstość punktów: największe skupiska punktów znajdują się wokół `(0, 0)` – czyli w pobliżu średniej wartości cech. Można zauważyć, że dane są dość mocno skoncentrowane w tym obszarze, a rzadziej występują na krańcach.
- Potencjalna korelacja: widoczna jest lekka skośność (pozytywna korelacja) między cechami – im większa cecha 1, tym częściej cecha 2 również wzrasta.

## 6. Definicja modelu regresji logistycznej w PyTorch

Tworzymy klasę `LR`, czyli prosty model regresji logistycznej. Składa się z jednej warstwy liniowej `Linear` przyjmującej `n_features` i zwracającej wartość prawdopodobieństwa. Na wyjściu stosujemy funkcję sigmoidalną, by uzyskać wynik w przedziale (0, 1) w ramach klasycznej regresji logistycznej binarnej.

## 7. Inicjalizacja modelu, optymalizatora i funkcji straty

Tworzymy instancję modelu, przekazując liczbę cech wejściowych, optymalizator: `SGD` (stochastyczny spadek gradientu) z learning rate = 1, funkcję straty: `BCELoss` do klasyfikacji binarnej. Ustawiamy liczbę epok treingu na 5.

## 8. Funkcja treningu modelu i jego trenowanie.

Pętla treningowa wykonuje się przez wcześniej ustaloną liczbę epok (zmienna `EPOCHS`). W każdej epoce:
- liczymy forward pass
- obliczamy stratę
- backpropagation
- aktualizujemy wagi

Na koniec wypisujemy stratę dla każdej epoki. Następnie trenujemy model na danych treningowych przy użyciu wcześniej zdefiniowanej funkcji `train`.

## 9. Pomiar dokładności na danych testowych

Funkcja `accuracy` sprawdza, czy model dobrze klasyfikuje dane. Wyniki wyjściowe `out` traktowane są jako przewidywania klasy 1, jeśli są w odległości < 0.5 od 1. Następnie obliczamy  średnią liczbę poprawnych przewidywań. Na koniec wypisujemy dokładność na zbiorze testowym.

#### Obserwacje wyników

- Strata systematycznie spada z każdą epoką, co oznacza, że model uczy się poprawnie.
- Spadek jest największy na początku, między epoką 1 a 2, a potem się stabilizuje.
- Ostateczna wartość straty (≈ 0.13) sugeruje, że model dobrze dopasował się do danych treningowych.
- Model osiągnął bardzo wysoką trafność klasyfikacji: 97.87%. To wskazuje na brak under- i overfittingu. Wynik jest bardzo dobry jak na prosty model logistyczny działający tylko na standaryzowanych danych bez rozszerzeń cech.

## 10. Definicja modelu EncryptedLR_eval

Tworzmy klasę `EncryptedLR_eval`, która odwzorowuje działanie modelu regresji logistycznej w środowisku zaszyfrowanym. Metoda `encrypt(context)` zamienia wagi i bias na zaszyfrowane wektory CKKS. Funkcja `forward()` działa na zaszyfrowanych danych wejściowych przy pomocy operacji dopuszczalnych w szyfrowaniu homomorficznym; `decrypt()` umożliwia odszyfrowanie wag i biasów.

## 11. Inicjalizacja modelu i konfiguracja kontekstu szyfrowania CKKS oraz szyfrowanie danych testowych

Tworzymy instancję `EncryptedLR_eval` na podstawie wytrenowanego modelu. Konfigurujemy kontekst CKKS z parametrami:

- `poly_mod_degree = 4096` – kompromis między wydajnością a dokładnością.
- `coeff_mod_bit_sizes = [40, 20, 40]`– rozmiary poziomów w drzewie resztowym.
- `global_scale = 2²⁰` – zapewnia dobrą precyzję i stabilność operacji.

Galois keys są wymagane do operacji takich jak rotacje lub mnożenia przez skalary.

Dane testowe są szyfrowane w formie wektorów CKKS – jeden wektor na jedną próbkę. Pomiar czasu pokazuje koszt szyfrowania, który może być znaczny dla dużych zbiorów danych.

## 12. Funkcja ewaluacji zaszyfrowanego modelu uruchomienie jej i porównanie z klasyczną

Funkcja wykonuje forward pass na zaszyfrowanych danych testowych. Po każdej predykcji następuje odszyfrowanie i klasyfikacja przy użyciu funkcji sigmoidalnej. Porównuje się wynik z prawdziwą etykietą, i zlicza poprawne trafienia. Zwracamy łączną dokładność oraz czas ewaluacji.

Uruchamiamy ewaluację na zaszyfrowanym zbiorze danych i porównujemy wynik z klasyczną dokładnością `plain_accuracy`. Jeśli przypadkiem trafność zaszyfrowana byłaby wyższa (co jest możliwe np. przez zaokrąglenia), kod to zauważy i wypisze komunikat.

#### Obserwacje wyników:

- Proces szyfrowania danych testowych był błyskawiczny, czas poniżej 1s.
- Ewaluacja odbyła się bardzo szybko – tylko 1 sekunda dla 188 próbek.
- rafność (accuracy) modelu na zaszyfrowanych danych wynosi 88.83%. Jest to nieco niższy wynik niż dla danych jawnych, jednak wciąż przyzwoity szczególnie jak na obliczenia szyfrowane.
- Różnica trafności to ~9.04 p.p., czyli zaszyfrowany model klasyfikuje poprawnie średnio 9% mniej przypadków niż model działający na danych jawnych. To akceptowalna cena za zachowanie prywatności, zwłaszcza w zastosowaniach takich jak: ochrona danych medycznych czy klasyfikacja danych finansowych.

## 13. Bezpieczna ewaluacja z użyciem TenSEAL – ograniczenia i zalecenia

Biblioteka TenSEAL (tak jak wiele innych bibliotek FHE) używa obiektów takich jak `CKKSVector`, które posiadają wewnętrzny stan napisany w języku C++. Obiekty te nie są bezpieczne w środowiskach wielowątkowych, co oznacza, że próby równoległego przetwarzania z użyciem współdzielonej pamięci (np. threading) mogą prowadzić do błędów, awarii lub uszkodzenia danych.

Dodatkowo, obiekty te nie mogą być serializowane (pickle), co uniemożliwia ich przekazywanie do procesów potomnych w klasycznym modelu multiprocessing.

Zamiast współdzielenia danych zaszyfrowanych między wątkami lub procesami, należy zastosować równoległość opartą na procesach (process-based parallelism), gdzie każdy proces tworzy swój własny kontekst TenSEAL i szyfruje dane niezależnie.

## 14. Porównanie sekwencyjnego i równoległego szyfrowania oraz ewaluacja zaszyfrowanego modelu

Skrypt wykonuje wielokrotne porównanie dwóch podejść: sekwencyjnego szyfrowania i ewaluacji oraz równoległego z użyciem multiprocessingu przy wykorzystaniu lokalnych kontekstów TenSEAL.

Importujemy bibliotekę potrzebną multiprocessingu, funkcje `encrypt_chunk` i `evaluate_chunk`, które obsługują szyfrowanie i ewaluację w workerach. Zapewniamy kompatybilność z bibliotekami FHE (np. TenSEAL), które nie są bezpieczne przy domyślnej metodzie fork.

Następnie ustalamy parametry i inicjalizujemy zmienne:

- `num_procs`: liczba procesów równoległych.

- `num_runs`: liczba powtórzeń eksperymentu, by uśrednić wyniki.

Inicjalizujemy listy do zapisu czasów i trafności w obu wariantach.

## 15. Konfiguracja kontekstu TenSEAL i modelu

Tworzymy kontekst CKKS z wybranymi parametrami. Model `EncryptedLR_eval` jest inicjalizowany na podstawie wcześniej wytrenowanego modelu PyTorch. Wszystkie obiekty są serializowane, aby mogły być bezpiecznie przesłane do workerów.

## 16. Przygotowanie danych, szyfrowanie sekwencyjne i równoległe

Dane testowe są konwertowane do formatu NumPy, aby mogły być dzielone na kawałki i przesyłane do procesów. 
##### Sekwencyjne szyfrowanie: 
Każdy wiersz danych testowych jest zamieniany na `CKKSVector`, zaszyfrowany i zserializowany. Proces ten odbywa się jeden po drugim. 
##### Równoległe szyfrowanie:
Dane testowe są dzielone na `num_procs` części. Każdy worker szyfruje fragment danych własnym lokalnym kontekstem. Wszystkie zaszyfrowane fragmenty są zbierane w jedną listę.

## 17. Ewaluacja zaszyfrowanych danych

##### Sekwencyjna:
Każdy zaszyfrowany wektor jest deserializowany i podawany na wejście modelu. Wynik jest odszyfrowywany, przekształcany przez funkcję sigmoidalną i porównywany z prawdziwą etykietą.

##### Równoległa:
Zaszyfrowane dane są dzielone i przekazywane do workerów razem z etykietami. Każdy worker wykonuje ewaluację lokalnie i zwraca wyniki do głównego procesu.





## 18. Zebranie wyników, zapisanie i porównanie metryk, wizualizacja

Czas i trafność są zapisywane w odpowiednich listach, obliczane są przyspieszenie czasowe i różnice trafności. Tworzone są dwa wykresy porównujące: czas całkowity w każdej próbie, trafność modelu w każdej próbie.

##### Obserwacje:

We wszystkich przypadkach trafność modelu jest identyczna lub różni się nieznacznie, co świadczy o stabilności algorytmu przy równoległym podejściu.

W każdym z trzech przebiegów podejście sekwencyjne było znacznie szybsze niż równoległe. Czas działania rośnie dla obu metod wraz z kolejnymi uruchomieniami, ale dla podejścia równoległego wzrost ten jest znacznie bardziej stromy (np. ~13s → ~20s). Podejście sekwencyjne oscyluje w granicach 3 sekund i rośnie minimalnie.

Równoległe szyfrowanie i ewaluacja nie przynoszą korzyści czasowych, wręcz są mniej wydajne w obecnej konfiguracji i skali. Jednak w przypadku większych zbiorów danych podejście równoległe ma większy wpływ na czas obliczeń.

## 19. Testy z wykorzystaniem sklearn

W tej wersji projektu zastąpiono model regresji logistycznej zaimplementowany w PyTorch jego odpowiednikiem z biblioteki `Scikit-Learn`. Zachowany został ten sam zbiór danych `Breast Cancer Dataset`, jednak teraz przygotowano go w formacie kompatybilnym z sklearn, co umożliwia bezpośrednie wykorzystanie funkcji takich jak `LogisticRegression`, `train_test_split`, `StandardScaler`, itd. Jednocześnie testowano ewaluację na danych zaszyfrowanych homomorficznie (FHE, CKKS – TenSEAL), zarówno w wersji sekwencyjnej, jak i równoległej.

## 20. Podsumowanie danych

Zastosowano podział 2:1 (trening:test), czyli ~67% danych do treningu, ~33% do testu. Dane zawierają 54 cechy.

## 21. Obserwacje wynikow z sklearn

- Równoległość przyniosła realne przyspieszenie, czas działania skrócił się blisko czterokrotnie dzięki wykorzystaniu multiprocessing. To duża różnica względem wcześniejszych wyników z PyTorch, gdzie równoległość była wolniejsza.
- Zarówno wersja sekwencyjna, jak i równoległa osiągały dokładnie 63.97% trafności na zbiorze testowym. Oznacza to, że szyfrowanie i sposób przetwarzania nie miały negatywnego wpływu na skuteczność predykcji.
- Trafność niższa niż w wersji nieszyfrowanej. Wcześniej model na danych jawnych osiągał 76.52% – różnica ok. 12.5 punktu procentowego.


Równoległa ewaluacja homomorficzna nie wpływa negatywnie na skuteczność klasyfikacji. Brak rozrzutu sugeruje wysoką powtarzalność działania modelu i stabilność FHE.

## 22. PyTorch vs Sklearn

##### Zalety PyTorch:
- Bardziej elastyczny, możesz dokładnie kontrolować architekturę (np. sigmoid, warstwy).
- Lepsza jakość modelu przy mniejszym zbiorze (prawie 98% accuracy).
- Niższy czas ewaluacji zaszyfrowanej, ale na znacznie mniejszych danych.

#####  Zalety Scikit-learn:

- Szybszy i prostszy w użyciu dla klasycznych problemów.
- Umożliwia użycie pełnego zbioru i regularizacji bez dodatkowego kodu.
- Świetnie współpracuje z równoległą ewaluacją FHE (speedup 3.6x przy stałej accuracy).

##### Kiedy wybrać Sklearn a kiedy PyTorch:

- Pełna kontrola nad modelem, niestandardowa architektura - PyTorch
- Trenować duży zbiór i porównać wiele modeli - Sklearn
- Łączyć klasyczne ML z deep learningiem - PyTorch
- Użyć gotowej regresji z automatycznym skalowaniem i regularyzacją - Sklearn